In [17]:
workspace()

In [18]:
using EvolvingGraphs
using EvolvingGraphs.Centrality

# Katz Motivation exmple 

In [2]:
g = EvolvingGraph{Node{String}, Int}()

Directed EvolvingGraph 0 nodes, 0 static edges, 0 timestamps

In [3]:
add_bunch_of_edges!(g, [("A", "B", 1), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 3)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [4]:
edges(g)

5-element Array{EvolvingGraphs.WeightedTimeEdge{EvolvingGraphs.Node{String},Int64,Float64},1}:
 Node(A)-1.0->Node(B) at time 1
 Node(A)-1.0->Node(C) at time 2
 Node(A)-1.0->Node(B) at time 2
 Node(C)-1.0->Node(A) at time 2
 Node(B)-1.0->Node(C) at time 3

In [6]:
katz(g)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.742301)
 (Node(B), 0.42943) 
 (Node(C), 0.514373)

In [8]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 3), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [9]:
katz(g2)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

In [17]:
g3 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g3, [("A", "B", 100), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])
katz(g3)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

In [11]:
g1 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g1, [("A","B", 1),("B","C", 2)])
katz(g1)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.64654) 
 (Node(B), 0.604677)
 (Node(C), 0.465136)

In [14]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 3), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])
katz(g2)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

# Katz Centrality

Each node `v` has forword neighbours. Each forward neighbour has forword neighbour, and so on.

In [7]:
"""
distance between two active nodes

alpha: time unit parameter
"""
function temporal_distance(v1::TimeNode, v2::TimeNode, beta::Real = 1.)
    return beta* abs(node_timestamp(v1) - node_timestamp(v2))
end
function temporal_distance(v1::Tuple{Int,Int}, v2::Tuple{Int,Int}, beta::Real = 1.)
    return beta * abs(v1[2] - v2[2])
end



temporal_distance (generic function with 4 methods)

In [7]:
function temporal_katz(g::AbstractEvolvingGraph, start::TimeNode; alpha = 0.1, k = 5)
    score = 0.
    v = start
    fronter = [v]
    level = 0
    while level < k
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                push!(next, v)
                td = temporal_distance(start, u)
                d = td + level
                score += alpha^d
            end
        end
        fronter = next
        level += 1
    end
    return score
end

function temporal_katz(g::IntAdjacencyList, start::Tuple{Int,Int}; alpha = 0.1, k = 5)
    score = 0.
    v = start
    fronter = [v]
    level = 0
    while level < k
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                push!(next, v)
                td = temporal_distance(start, u)
                d = td + level
                score += alpha^d
            end
        end
        fronter = next
        level += 1
    end
    return score
end

# Can be done in parallel, focus on useful node at certain time stamp


temporal_katz (generic function with 2 methods)

In [9]:
active_nodes(g)

7-element Array{EvolvingGraphs.TimeNode{String,Int64},1}:
 TimeNode(A, 1)
 TimeNode(B, 1)
 TimeNode(A, 2)
 TimeNode(C, 2)
 TimeNode(B, 2)
 TimeNode(B, 3)
 TimeNode(C, 3)

In [8]:
an = active_nodes(g)

scores = Dict()
for n in an
    scores[n] = temporal_katz(g, n, alpha = 0.1, k = 5)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(A, 1) => 2.22435
  TimeNode(A, 2) => 2.32421
  TimeNode(B, 1) => 2.0111
  TimeNode(B, 2) => 1.01
  TimeNode(B, 3) => 1.0
  TimeNode(C, 2) => 2.2324
  TimeNode(C, 3) => 0.0

In [21]:
using DataStructures

sum_scores = DefaultDict(0.) 
for (n, v) in scores
    sum_scores[node_key(n)] += v
end
sum_scores 

DataStructures.DefaultDict{Any,Any,Float64} with 3 entries:
  "B" => 4.11132
  "A" => 4.2323
  "C" => 2.33643

In [15]:
an = active_nodes(g2)

scores = Dict()
for n in an
    scores[n] = temporal_katz(g2, n, alpha = 0.1, k = 5)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(A, 2) => 3.2424
  TimeNode(B, 3) => 0.0
  TimeNode(B, 1) => 3.11132
  TimeNode(C, 1) => 1.01324
  TimeNode(A, 3) => 1.0
  TimeNode(C, 2) => 1.32421
  TimeNode(B, 2) => 1.0

In [20]:
an = active_nodes(g3)

scores = Dict()
for n in an
    scores[n] = temporal_katz(g3, n, alpha = 0.1, k = 5)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(C, 2)   => 1.3232
  TimeNode(B, 100) => 0.0
  TimeNode(B, 1)   => 3.11132
  TimeNode(C, 1)   => 1.01323
  TimeNode(B, 2)   => 1.0
  TimeNode(A, 100) => 1.0
  TimeNode(A, 2)   => 3.2323

# Communicability Betweenness Centrality 

In [13]:
"""
Take account of all forward neighours.
"""
function temporal_katz_with_v_removed(g::EvolvingGraph, v; alpha = 0.1, k = 5)
    score = 0.
    fronter = [v]
    level = 0
    start = v
    while level < k
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                if v != start
                    push!(next, v)
                    d = temporal_distance(v, u)
                    d += level
                    score += alpha^d
                end
            end
        end
        fronter = next
        level += 1
    end
    return score
end



function temporal_communicability(g::EvolvingGraph, v::TimeNode; alpha = 0.1, k = 5)
    
    score = temporal_katz(g, v, alpha = alpha, k = k)
    score_with_v_removed = temporal_katz_with_v_removed(g, v, alpha = alpha, k = k)
    
    r = (score - score_with_v_removed)/ score
    isnan(r)? 0 : r
end

temporal_communicability (generic function with 1 method)

In [14]:
an = active_nodes(g)

scores = Dict()
for n in an
    scores[n] = temporal_communicability(g, n, alpha = 0.1, k = 100)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 3) => 0.0
  TimeNode(A, 2) => 0.0564789
  TimeNode(C, 3) => 0
  TimeNode(A, 1) => 0.0
  TimeNode(B, 2) => 0.0
  TimeNode(B, 1) => 0.0
  TimeNode(C, 2) => 0.0860369

In [12]:
0/0

NaN

# Temporal Betweenness

In constrast with Katz and Communicability (which are based on walks), the following centrality algorithms are based on shortest paths.

In [ ]:
#TODO later

# Temporal Closeness Centrality 

Closeness centrality (or closeness) of a node is a measure of centrality in a network, calculated as the sum of the length of the shortest paths between the node and all other nodes in the graph.

In [18]:
function temporal_closeness(g::Union{AbstractEvolvingGraph, IntAdjacencyList}, start::Union{TimeNode,Tuple{Int,Int}})
    v = start
    level = Dict(v => 0)
    i = 1
    fronter = [v]
    while length(fronter) > 0
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                if !(v in keys(level))
                    td = temporal_distance(start, u)
                    level[v] = i + td
                    push!(next, v)
                end
            end
        end
        fronter = next
        i += 1
    end
    
    total_scores = sum(values(level))
    return total_scores > 0. ? (length(level) - 1)/total_scores : 0.
end

temporal_closeness (generic function with 1 method)

In [16]:
edges(g)

5-element Array{EvolvingGraphs.WeightedTimeEdge{EvolvingGraphs.Node{String},Int64,Float64},1}:
 Node(A)-1.0->Node(B) at time 1
 Node(A)-1.0->Node(C) at time 2
 Node(A)-1.0->Node(B) at time 2
 Node(C)-1.0->Node(A) at time 2
 Node(B)-1.0->Node(C) at time 3

In [19]:
an = active_nodes(g)

scores = Dict()
for n in an
    scores[n] = temporal_closeness(g, n)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(A, 1) => 0.344828
  TimeNode(A, 2) => 0.625
  TimeNode(C, 2) => 0.416667
  TimeNode(C, 3) => 0.0
  TimeNode(B, 1) => 0.5
  TimeNode(B, 2) => 0.5
  TimeNode(B, 3) => 1.0

# Temporal PageRank

Random walk interpretation can not travel back in time.

* Block matrix version (use reverse pagerank): useful for evolving graph with a small number of timestamps

* Probability based (time-stamps based rating): Random walk interpretation. Not fixed probability for all nodes but different probability for each individual node. 

In [48]:
function block_google_matrix(g::IntAdjacencyList; alpha = 0.85, balance = 0.75)
    A = full(block_adjacency_matrix(g))
    AT = A.T
    N, N = size(A)
    p = ones(N)/N
    dangling_weights = p
    dangling_nodes = find(x->x==0, sum(A,2))
    for node in dangling_nodes
        A[node,:] = dangling_weights
    end
    A ./= sum(A,2)
    return alpha * A .+ (1- alpha) * p
end

function google_matrix(g::AbstractEvolvingGraph)
    
end

google_matrix (generic function with 1 method)

In [49]:
g = IntAdjacencyList(4,3)
add_edge!(g, 1, 2, 1)
add_edge!(g, 2, 3, 1)
add_edge!(g, 1, 4, 2)

Directed IntAdjacencyList (4 nodes, 3 static edges, 3 timestamps)

In [57]:
A = google_matrix_block(g)
sum(A, 2)

12×1 Array{Float64,2}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [60]:
function temporal_pagerank(g::AbstractEvolvingGraph)
    A = block_google_matrix(g)
    F = eigfact(A')
    println("F values $(F[:values])")
    return F[:vectors][:,4]
end

temporal_pagerank (generic function with 1 method)

In [61]:
temporal_pagerank(g)

F values Complex{Float64}[-0.10625+0.221177im, -0.10625-0.221177im, -6.49827e-16+0.0im, 1.0+0.0im, -1.46263e-17+7.29755e-10im, -1.46263e-17-7.29755e-10im, 3.35485e-18+0.0im, 1.38933e-17+0.0im, 0.0+0.0im, 4.53455e-33+0.0im, 0.0+0.0im, 0.0+0.0im]


12-element Array{Complex{Float64},1}:
 0.213978+0.0im
 0.304918+0.0im
 0.473158+0.0im
 0.213978+0.0im
 0.304918+0.0im
 0.213978+0.0im
 0.213978+0.0im
 0.473158+0.0im
 0.213978+0.0im
 0.213978+0.0im
 0.213978+0.0im
 0.213978+0.0im

# JMLR and Random Evolving Graph Data

Start with the original definition and compare the change of ranking of top authors if we vary

In [17]:
g = EvolvingGraph()
g = random_evolving_graph(g, 100, 5, 0.2)

Directed EvolvingGraph 100 nodes, 9771 static edges, 5 timestamps

In [18]:
r = katz(g)
sorted_top = sort(r, by = x -> x[2], rev = true)[1:20]

20-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(32), 0.320363) 
 (Node(58), 0.244402) 
 (Node(100), 0.235876)
 (Node(2), 0.199925)  
 (Node(74), 0.198615) 
 (Node(83), 0.162692) 
 (Node(26), 0.152276) 
 (Node(97), 0.138456) 
 (Node(68), 0.137589) 
 (Node(18), 0.126655) 
 (Node(7), 0.120336)  
 (Node(57), 0.108358) 
 (Node(10), 0.103718) 
 (Node(44), 0.099909) 
 (Node(45), 0.08922)  
 (Node(87), 0.0857851)
 (Node(33), 0.0857679)
 (Node(88), 0.0825459)
 (Node(11), 0.0791009)
 (Node(65), 0.0738873)

In [19]:
sorted_top_keys = map(x -> node_key(x[1]), sorted_top);

In [20]:
intg = evolving_graph_to_adj(g)

Directed IntAdjacencyList (100 nodes, 9771 static edges, 5 timestamps)

In [23]:
scores = []
for n in active_nodes(g)
    if node_key(n) in sorted_top_keys
        intn = (node_key(n),node_timestamp(n))
        println(intn)
        tr = temporal_katz(intg, intn, alpha = 0.1, k = 5)
        push!(scores, (n, tr))
    end
end
scores

(33, 1)
(83, 1)
(87, 1)
(100, 1)
(2, 1)
(11, 1)
(26, 1)
(58, 1)
(74, 1)
(18, 1)
(65, 1)
(10, 1)
(32, 1)
(68, 1)
(44, 1)
(45, 1)
(7, 1)
(88, 1)
(97, 1)
(57, 1)
(2, 2)
(26, 2)
(32, 2)
(33, 2)
(45, 2)
(68, 2)
(87, 2)
(7, 2)
(10, 2)
(18, 2)
(74, 2)
(11, 2)
(58, 2)
(100, 2)
(57, 2)
(65, 2)
(83, 2)
(44, 2)
(88, 2)
(97, 2)
(10, 3)
(100, 3)
(2, 3)
(32, 3)
(44, 3)
(7, 3)
(18, 3)
(74, 3)
(26, 3)
(45, 3)
(97, 3)
(33, 3)
(83, 3)
(88, 3)
(87, 3)
(57, 3)
(58, 3)
(11, 3)
(65, 3)
(68, 3)
(2, 4)
(11, 4)
(32, 4)
(65, 4)
(74, 4)
(87, 4)
(97, 4)
(10, 4)
(45, 4)
(57, 4)
(18, 4)
(83, 4)
(88, 4)
(7, 4)
(26, 4)
(100, 4)
(44, 4)
(33, 4)
(58, 4)
(68, 4)
(7, 5)
(26, 5)
(83, 5)
(2, 5)
(33, 5)
(45, 5)
(74, 5)
(44, 5)
(58, 5)
(100, 5)
(65, 5)
(87, 5)
(97, 5)
(57, 5)
(68, 5)
(88, 5)
(18, 5)
(10, 5)
(11, 5)
(32, 5)


100-element Array{Any,1}:
 (TimeNode(33, 1), 533.44)  
 (TimeNode(83, 1), 455.689) 
 (TimeNode(87, 1), 499.32)  
 (TimeNode(100, 1), 662.418)
 (TimeNode(2, 1), 981.951)  
 (TimeNode(11, 1), 756.512) 
 (TimeNode(26, 1), 441.032) 
 (TimeNode(58, 1), 738.887) 
 (TimeNode(74, 1), 519.039) 
 (TimeNode(18, 1), 708.973) 
 (TimeNode(65, 1), 469.141) 
 (TimeNode(10, 1), 738.041) 
 (TimeNode(32, 1), 687.148) 
 ⋮                          
 (TimeNode(58, 5), 638.927) 
 (TimeNode(100, 5), 337.539)
 (TimeNode(65, 5), 669.189) 
 (TimeNode(87, 5), 647.304) 
 (TimeNode(97, 5), 590.473) 
 (TimeNode(57, 5), 877.297) 
 (TimeNode(68, 5), 853.998) 
 (TimeNode(88, 5), 751.361) 
 (TimeNode(18, 5), 502.457) 
 (TimeNode(10, 5), 424.658) 
 (TimeNode(11, 5), 699.687) 
 (TimeNode(32, 5), 706.903) 

In [25]:
using DataStructures

sum_scores = DefaultDict(0.) 
for (n, v) in scores
    sum_scores[node_key(n)] += v
end
sum_scores;

In [31]:
accu_scores = DefaultDict([0.]) 
for (n, v) in scores
#     println("node $n")
#     println("value $v")
    push!(accu_scores[node_key(n)], v)
#     push!(accu_scores[node_key(n)], accu_scores[node_key(n)][end] + v)
end
accu_scores

node TimeNode(11, 1)
value 1224.834899819936
node TimeNode(17, 1)
value 1163.8518998292761
node TimeNode(45, 1)
value 1293.5838998093886
node TimeNode(52, 1)
value 906.0639998690834
node TimeNode(25, 1)
value 778.2052998885803
node TimeNode(34, 1)
value 1238.168899817753
node TimeNode(36, 1)
value 1539.8606997719116
node TimeNode(54, 1)
value 1144.227099832334
node TimeNode(55, 1)
value 1169.1183998284694
node TimeNode(43, 1)
value 1099.1326998390223
node TimeNode(80, 1)
value 1363.896299798577
node TimeNode(23, 1)
value 1047.9910998470273
node TimeNode(6, 1)
value 1296.9582998088517
node TimeNode(37, 1)
value 1228.5242998193146
node TimeNode(46, 1)
value 910.8328998682736
node TimeNode(60, 1)
value 1141.6199998327547
node TimeNode(89, 1)
value 1341.6504998016464
node TimeNode(33, 1)
value 922.5920998664808
node TimeNode(73, 1)
value 900.4506998700047
node TimeNode(94, 1)
value 1128.5436998347964
node TimeNode(11, 2)
value 746.1284998952133
node TimeNode(17, 2)
value 910.4348998706787


DataStructures.DefaultDict{Any,Any,Array{Float64,1}} with 0 entries

In [30]:
using Plots

INFO: Precompiling module Plots.


In [26]:
r2 = collect(sum_scores);

In [27]:
sorted_r2 = sort(r2, by = x -> x[2], rev = true)

20-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}(2, 3596.32)  
 Pair{Any,Any}(58, 3370.67) 
 Pair{Any,Any}(7, 3226.47)  
 Pair{Any,Any}(11, 3048.78) 
 Pair{Any,Any}(88, 3047.22) 
 Pair{Any,Any}(18, 2958.37) 
 Pair{Any,Any}(57, 2957.42) 
 Pair{Any,Any}(65, 2942.76) 
 Pair{Any,Any}(32, 2939.11) 
 Pair{Any,Any}(97, 2930.67) 
 Pair{Any,Any}(33, 2926.23) 
 Pair{Any,Any}(45, 2879.59) 
 Pair{Any,Any}(87, 2832.12) 
 Pair{Any,Any}(10, 2771.23) 
 Pair{Any,Any}(100, 2756.39)
 Pair{Any,Any}(68, 2746.95) 
 Pair{Any,Any}(44, 2738.5)  
 Pair{Any,Any}(83, 2672.8)  
 Pair{Any,Any}(74, 2669.7)  
 Pair{Any,Any}(26, 2452.59) 

In [28]:
sorted_top

20-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(32), 0.320363) 
 (Node(58), 0.244402) 
 (Node(100), 0.235876)
 (Node(2), 0.199925)  
 (Node(74), 0.198615) 
 (Node(83), 0.162692) 
 (Node(26), 0.152276) 
 (Node(97), 0.138456) 
 (Node(68), 0.137589) 
 (Node(18), 0.126655) 
 (Node(7), 0.120336)  
 (Node(57), 0.108358) 
 (Node(10), 0.103718) 
 (Node(44), 0.099909) 
 (Node(45), 0.08922)  
 (Node(87), 0.0857851)
 (Node(33), 0.0857679)
 (Node(88), 0.0825459)
 (Node(11), 0.0791009)
 (Node(65), 0.0738873)

In [20]:
using EvolvingGraphs
using EvolvingGraphs.Centrality
import EzXML


# load graph data at each year
# form evolving graph
g = EvolvingGraph{Node{String}, Int}()

for year in range(2001, 17)
    sg = load_graphml("jmlr_$(year).graphml")
    add_graph!(g, sg, year)
end
g

Directed EvolvingGraph 3354 nodes, 15654 static edges, 17 timestamps

In [6]:
rating = katz(g, 0.1, 0.01; mode = :receive)


sorted_authors = sort(rating, by = x -> x[2], rev = true)

println("Top 10 rating authors")
println(sorted_authors[1:10])

println("Button 10 rating authors")
println(sorted_authors[end-10:end])

Top 10 rating authors
Tuple{EvolvingGraphs.Node{String},Float64}[(Node(Manuel Gomez-Rodriguez), 1.0), (Node(Henry Adams), 0.984369), (Node(Joseph Bradley), 0.972919), (Node(Le Song), 0.959548), (Node(Wei-Sheng Chin), 0.937996), (Node(Zoubin Ghahramani), 0.891851), (Node(Lori Ziegelmeier), 0.885932), (Node(Francis Motta), 0.885932), (Node(Eric Hanson), 0.885932), (Node(Tegan Emerson), 0.885932)]
Button 10 rating authors
Tuple{EvolvingGraphs.Node{String},Float64}[(Node(Sa, Virginia R. de), 2.49324e-5), (Node(Chapados, Nicolas), 2.49324e-5), (Node(Forman, George), 2.49324e-5), (Node(Pla, Ferran), 2.49299e-5), (Node(Gavinsky, Dmitry), 2.49299e-5), (Node(Zhang, Huajie), 2.49299e-5), (Node(Wrobel, Stefan), 2.49299e-5), (Node(Vempala, Santosh), 2.49299e-5), (Node(Veloso, Manuela), 2.49299e-5), (Node(Struyf, Jan), 2.49299e-5), (Node(Barto, Andrew G.), 2.49299e-5)]


In [37]:
an = active_nodes(g)[1:11]

scores = Dict()
for n in an
    scores[n] = temporal_katz(g, n, alpha = 0.1, k = 5)
end
scores

start node TimeNode(Manevitz, Larry M., 2001)
at level 0, with 1 nodes
at level 1, with 2 nodes
at level 2, with 3 nodes
at level 3, with 5 nodes
at level 4, with 8 nodes
start node TimeNode(Yousef, Malik, 2001)
at level 0, with 1 nodes
at level 1, with 1 nodes
at level 2, with 2 nodes
at level 3, with 3 nodes
at level 4, with 5 nodes
start node TimeNode(Gates, Kevin E., 2001)
at level 0, with 1 nodes
at level 1, with 2 nodes
at level 2, with 6 nodes
at level 3, with 19 nodes
at level 4, with 62 nodes
start node TimeNode(Masters, Annette, 2001)
at level 0, with 1 nodes
at level 1, with 2 nodes
at level 2, with 6 nodes
at level 3, with 19 nodes
at level 4, with 62 nodes
start node TimeNode(Downs, Tom, 2001)
at level 0, with 1 nodes
at level 1, with 4 nodes
at level 2, with 13 nodes
at level 3, with 43 nodes
at level 4, with 145 nodes
start node TimeNode(Vapnik, Vladimir, 2001)
at level 0, with 1 nodes
at level 1, with 3 nodes
at level 2, with 11 nodes
at level 3, with 40 nodes
at level 

Dict{Any,Any} with 11 entries:
  TimeNode(Manevitz, Larry M., 2001)  => 2.3593
  TimeNode(Singer, Yoram, 2001)       => 4.86033
  TimeNode(Downs, Tom, 2001)          => 5.9246
  TimeNode(Vapnik, Vladimir, 2001)    => 4.6955
  TimeNode(Crammer, Koby, 2001)       => 6.33534
  TimeNode(Horn, David, 2001)         => 4.6955
  TimeNode(Masters, Annette, 2001)    => 2.8727
  TimeNode(Ben-Hur, Asa, 2001)        => 7.7474
  TimeNode(Yousef, Malik, 2001)       => 1.2358
  TimeNode(Gates, Kevin E., 2001)     => 2.8727
  TimeNode(Siegelmann, Hava T., 2001) => 4.6955